In [3]:
import warnings
from tensorflow import keras
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
from sklearn.metrics import confusion_matrix
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import imagenet_utils
from tensorflow.keras.layers import Activation, Dense, GlobalAveragePooling2D
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator

tf.compat.v1.reset_default_graph()
warnings.simplefilter(action='ignore', category=FutureWarning)


2024-01-24 19:10:31.245209: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-24 19:10:34.302349: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-24 19:10:34.302580: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-24 19:10:35.112950: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-24 19:10:36.343807: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-24 19:10:36.347542: I tensorflow/core/platform/cpu_feature_guard.cc:1

## Load and  Modify MobileNetV2 Model

In [4]:
# Download the MobileNet Model

mobile = tf.keras.applications.mobilenet.MobileNet(input_shape=(224, 224, 3))

2024-01-24 19:11:54.701322: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:274] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [5]:
def create_model(unit):
    x = mobile.layers[-2].output
    output = Dense(units=unit, activation='softmax')(x)
    model = Model(inputs=mobile.input, outputs=output)
    for layer in model.layers:
        layer.trinable = False

    return model

In [7]:
# Instantiate the Classifier Model (model) and the Breed and Age Classifier Model (model_1)
model = create_model(unit=15)
model_1 = create_model(unit=16)

## Data Preparation

### 1. Classifier

In [8]:
train_path ='/home/jm/Degree/4.2/project/project/classifier-data/train'
val_path = '/home/jm/Degree/4.2/project/project/classifier-data/val'
test_path = '/home/jm/Degree/4.2/project/project/classifier-data/test'

# Define the data generator
datagen = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input)

# Generate the image batches for the training, validation, and test datasets
train_batches = datagen.flow_from_directory(
    train_path,
    target_size=(224, 224),
    batch_size=16,
    class_mode='categorical'
)

val_batches = datagen.flow_from_directory(
    val_path,
    target_size=(224, 224),
    batch_size=16,
    class_mode='categorical',
    shuffle=False
)

test_batches = datagen.flow_from_directory(
    test_path,
    target_size=(224, 224),
    batch_size=38,
    class_mode='categorical',
    shuffle=False
)

Found 1538 images belonging to 15 classes.
Found 453 images belonging to 15 classes.
Found 218 images belonging to 15 classes.


### 2. Breed & Age Classifier

In [9]:
train_path_1 ='/home/jm/Degree/4.2/project/project/breed-age-data/data/train'
val_path_1 = '/home/jm/Degree/4.2/project/project/breed-age-data/data/val'
test_path_1 = '/home/jm/Degree/4.2/project/project/breed-age-data/data/test'

# Define the data generator
datagen = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input)

# Generate the image batches for the training, validation, and test datasets
train_batches_1 = datagen.flow_from_directory(
    train_path_1,
    target_size=(224, 224),
    batch_size=16,
    class_mode='categorical'
)

val_batches_1 = datagen.flow_from_directory(
    val_path_1,
    target_size=(224, 224),
    batch_size=16,
    class_mode='categorical',
    shuffle=False
)

test_batches_1 = datagen.flow_from_directory(
    test_path_1,
    target_size=(224, 224),
    batch_size=38,
    class_mode='categorical',
    shuffle=False
)

Found 854 images belonging to 16 classes.
Found 256 images belonging to 16 classes.
Found 265 images belonging to 16 classes.


## Train the Models

In [10]:
def train_fit(model, x, y, epoch=2):
    model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(x=x, validation_data=y, epochs=epoch, verbose=2)

In [ ]:
hist = train_fit(model=model,x=train_batches, y=val_batches)
hist_1 = train_fit(model=model_1,x=train_batches_1, y=val_batches_1)

Epoch 1/2


2024-01-24 19:12:39.167282: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 25690112 exceeds 10% of free system memory.
2024-01-24 19:12:40.110691: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 25690112 exceeds 10% of free system memory.
2024-01-24 19:12:40.309732: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 25690112 exceeds 10% of free system memory.
2024-01-24 19:12:40.338074: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 25690112 exceeds 10% of free system memory.
2024-01-24 19:12:40.359813: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 51380224 exceeds 10% of free system memory.
/home/jm/anaconda3/lib/python3.9/site-packages/PIL/Image.py:970: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


## Test the trained Models

### 1. Classifier Model

In [11]:
test_labels  = test_batches.classes
predictions = model.predict(x=test_batches, verbose=0)

# Evaluate on the test set
test_loss, test_accuracy = model.evaluate(test_batches)

# Print the test loss and accuracy
print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')

6/6 [==============================] - 7s 1s/step - loss: 0.6103 - accuracy: 0.8670
Test Loss: 0.6102837920188904
Test Accuracy: 0.8669725060462952


### 2. Breed & Age Classifier Model

In [12]:
test_labels_1  = test_batches_1.classes
predictions_1 = model_1.predict(x=test_batches_1, verbose=0)

# Evaluate on the test set
test_loss_1, test_accuracy_1 = model_1.evaluate(test_batches_1)

# Print the test loss and accuracy
print(f'Test Loss: {test_loss_1}')
print(f'Test Accuracy: {test_accuracy_1}')

7/7 [==============================] - 5s 745ms/step - loss: 0.2873 - accuracy: 0.9057
Test Loss: 0.2872687578201294
Test Accuracy: 0.9056603908538818


## Classes Definition

In [13]:
classes_1 = {0:'abyssinian horse',
 1:'adaev horse',
 2:'aegidienberger horse',
 3:'aging man', 4:'aging woman', 5:'american cat', 6:'boerboel dog', 7:'bulldog', 8:'chihuahua', 9:'egyptian mau cat',
 10:'german shepherd', 11:'infant', 12:'man',13:'sokoke cat', 14:'teenage boy', 15:'teenage girl'}
def class_pred(predictions):
    return classes_1[predictions]

In [14]:
classes = {
 0: "aeroplane",
 1:"ball", 2:"car", 3:"cat", 4:"dog", 5:'frog', 6:'girl', 7:'horse',
 8:'infant', 9:'laptop', 10:'man', 11:'puppy', 12:'river', 13:'track',
 14:'woman'}
def class_predicted(predictions):
    return classes[predictions]

## Predictions

In [15]:
# def make_predictions(img_path, model=model, model_1=model_1):


def make_predictions(img_path, model=model, model_1=model_1):

    classes_2 = ['cat', 'horse', 'dog']
    classes_3 = ['girl', 'man', 'woman']

    try:
        img = image.load_img(img_path, target_size=(224, 224))
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = tf.keras.applications.mobilenet.preprocess_input(img_array)

        # Make a prediction
        predictions = model.predict(img_array)

        # Get the predicted class index
        predicted_class_index = np.argmax(predictions[0])

        # Map the predicted class index to a label
        predicted_class_label = class_predicted(predicted_class_index)
        print(predicted_class_label)

        if predicted_class_label in classes_2:
            pred = model_1.predict(img_array)
            predicted_class_index_1 = np.argmax(pred[0])
            predicted_class_label_1 = class_pred(predicted_class_index_1)

            # Display the image with predicted class and breed
            img_rgb = image.img_to_array(img)
            img_rgb /= 255.0  # Normalize the image
            plt.imshow(img_rgb)
            plt.title(f"Predicted class: {predicted_class_label}\nBreed: {predicted_class_label_1}")
            plt.show()

        elif predicted_class_label in classes_3:
            pred = model_1.predict(img_array)
            predicted_class_index_1 = np.argmax(pred[0])
            predicted_class_label_1 = class_pred(predicted_class_index_1)

            # Display the image with predicted class and age group
            img_rgb = image.img_to_array(img)
            img_rgb /= 255.0  # Normalize the image
            plt.imshow(img_rgb)
            plt.title(f"Predicted class: {predicted_class_label}\nAge Group: {predicted_class_label_1}")
            plt.show()

        else:
            # Display the image with only predicted class
            img_rgb = image.img_to_array(img)
            img_rgb /= 255.0  # Normalize the image
            plt.imshow(img_rgb)
            plt.title(f"Predicted class: {predicted_class_label}")
            plt.show()

    except FileNotFoundError:
        print("Error: Image not loaded. Not available.")


In [2]:
make_predictions(img_path="/home/jm/Degree/4.2/project/project/classifier-data/abanera_3.jpg")

NameError: name 'make_predictions' is not defined

In [32]:
model.save('MobileNet_beta_FineTuned_Model.h5')


/home/jm/anaconda3/lib/python3.9/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [37]:
loaded_model = tf.keras.models.load_model('MobileNet_beta_FineTuned_Model.h5')